# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [267]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-82cb140b76-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Sport Workflow Series (Tennis explorative search") 

Consider the following exploratory information need:

> investigate tennis and find the main BGPs related to this sport. Compare players per countries, awards of the players and tournament.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q1426`| Roger Federer       | node |
| `wd:Q11459`| Serena Williams       | node |





Also consider

```
wd:Q1426 ?p ?obj .
```

is the BGP to retrieve all **properties of Roger Federer**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.



The workload should:


1. Identify the BGP for tennis player

2. Identify the BGP for tennis tournament

3. Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) (the result set must be a list of four couples Grand Slam IRI and label).

4. Find the three different type of possible surfaces in a tennis court (the result set must be a list of three couples surface IRI and label).

5. Return the tournaments played in the 2018 Wimbledon Championships edition with the related winners (hint: men and women do not play together, there are two distinct tournament ) (the result set must be a list of elements with tournament IRI and label, and winner IRI and label).

6. Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

7. Consider the country for sport property of the tennis player (NO Country of citizenship). Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

8. Consider the country for sport property of the tennis player (NO Country of citizenship). Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

## Task 1
Identify the BGP for tennis player

In [88]:
# query example
queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   wd:Q1426 ?p ?o .
   ?p sc:name ?pName .
   ?o sc:name ?oName .
   
}
LIMIT 120
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('pName', 'victory'), ('o', 'http://www.wikidata.org/entity/Q22956224'), ('oName', "2017 Australian Open – Men's Singles")]
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('pName', 'victory'), ('o', 'http://www.wikidata.org/entity/Q612597'), ('oName', '2007 Tennis Masters Cup')]
[('p', 'http://www.wikidata.org/prop/direct/P3342'), ('pName', 'significant person'), ('o', 'http://www.wikidata.org/entity/Q54584'), ('oName', 'Andy Roddick')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pName', 'award received'), ('o', 'http://www.wikidata.org/entity/Q56509472'), ('oName', 'honorary doctor of the University of Basel')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pName', 'participant in'), ('o', 'http://www.wikidata.org/entity/Q589310'), ('oName', "tennis at the 2000 Summer Olympics – men's singles")]
[('p', 'http://www.wikidata.org/prop/direct/P6758'), ('pName', 'supported sports team'), ('o', 'http://www.

Final query for this task

In [8]:
# write your final query
#Q10833314 tennis player
#P106 occupation
queryString = """
SELECT DISTINCT ?occupation ?occupationName
WHERE {
   wd:Q1426 wdt:P106 ?occupation .
   ?occupation sc:name ?occupationName .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('occupation', 'http://www.wikidata.org/entity/Q10833314'), ('occupationName', 'tennis player')]
1


## Task 2
Identify the BGP for tennis tournament

In [14]:
# write your queries
#Q22956224 2017 Australian Open – Men's Singles
queryString = """
SELECT DISTINCT ?s ?sName ?pName
WHERE {
    ?s ?p wd:Q22956224.
   ?p sc:name ?pName .
   ?s sc:name ?sName .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q22077628'), ('sName', '2017 Australian Open'), ('pName', 'has part')]
[('s', 'http://www.wikidata.org/entity/Q1426'), ('sName', 'Roger Federer'), ('pName', 'victory')]
[('s', 'http://www.wikidata.org/entity/Q46993124'), ('sName', "2018 Australian Open – Men's Singles"), ('pName', 'follows')]
3


In [15]:
#Q22077628 2017 Australian Open
queryString = """
SELECT DISTINCT ?pName ?o ?oName
WHERE {
   wd:Q22077628 ?p ?o .
   ?p sc:name ?pName .
   ?o sc:name ?oName .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('pName', 'follows'), ('o', 'http://www.wikidata.org/entity/Q21521385'), ('oName', '2016 Australian Open')]
[('pName', 'has part'), ('o', 'http://www.wikidata.org/entity/Q22956224'), ('oName', "2017 Australian Open – Men's Singles")]
[('pName', 'organizer'), ('o', 'http://www.wikidata.org/entity/Q223041'), ('oName', 'International Tennis Federation')]
[('pName', 'location'), ('o', 'http://www.wikidata.org/entity/Q2228393'), ('oName', 'Melbourne Park')]
[('pName', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q3141'), ('oName', 'Melbourne')]
[('pName', 'country'), ('o', 'http://www.wikidata.org/entity/Q408'), ('oName', 'Australia')]
[('pName', 'sport'), ('o', 'http://www.wikidata.org/entity/Q847'), ('oName', 'tennis')]
[('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q47345468'), ('oName', 'tennis tournament edition')]
[('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q60874'), ('oName', 'Australian Ope

In [17]:
#Q47345468 tennis tournament edition
queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   wd:Q47345468 ?p ?o .
   ?p sc:name ?pName .
   ?o sc:name ?oName .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P393'), ('oName', 'edition number')]
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('pName', 'has parts of the class'), ('o', 'http://www.wikidata.org/entity/Q47382471'), ('oName', 'day-by-day summary')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P580'), ('oName', 'start time')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P582'), ('oName', 'end time')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P17'), ('oName', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('o', 'http://www.wikidata.org/entity/Q13219666'), ('oName', 'tennis tournament')]
[('p', 'htt

In [20]:
#P361 part of
queryString = """
SELECT DISTINCT ?o ?oName
WHERE {
   wd:Q47345468 wdt:P361 ?o .
   ?o sc:name ?oName .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q13219666'), ('oName', 'tennis tournament')]
[('o', 'http://www.wikidata.org/entity/Q47358534'), ('oName', 'tennis tour edition')]
2


Final query for this task

In [22]:
# write your final query
queryString = """
SELECT DISTINCT ?o ?oName
WHERE {
   wd:Q47345468 wdt:P361 ?o .
   ?o sc:name ?oName .
}
LIMIT 1
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q13219666'), ('oName', 'tennis tournament')]
1


## Task 3
Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) (the result set must be a list of four couples Grand Slam IRI and label).

In [24]:
# write your queries
#Q13219666 tennis tournament

queryString = """
SELECT DISTINCT ?pName ?o ?oName
WHERE {
   wd:Q13219666 ?p ?o .
   ?p sc:name ?pName .
   ?o sc:name ?oName .
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q13406554'), ('oName', 'sports competition')]
[('pName', 'sport'), ('o', 'http://www.wikidata.org/entity/Q847'), ('oName', 'tennis')]
[('pName', 'has parts of the class'), ('o', 'http://www.wikidata.org/entity/Q46190676'), ('oName', 'tennis event')]
[('pName', "topic's main template"), ('o', 'http://www.wikidata.org/entity/Q6452600'), ('oName', 'Template:ATP Tournament')]
[('pName', 'has list'), ('o', 'http://www.wikidata.org/entity/Q6642846'), ('oName', 'list of tennis tournaments')]
[('pName', "topic's main template"), ('o', 'http://www.wikidata.org/entity/Q7474005'), ('oName', 'Template:Infobox tennis tournament year')]
[('pName', "topic's main template"), ('o', 'http://www.wikidata.org/entity/Q22803873'), ('oName', 'Template:WTA tournament')]
[('pName', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q5325422'), ('oName', 'Category:Tennis tournaments')]
[('pName', "topic's main template"), ('o'

In [31]:
#Q6642846 list of tennis tournaments
queryString = """
SELECT DISTINCT ?pName ?o ?oName
WHERE {
   wd:Q46190676 ?p ?o .
   ?p sc:name ?pName .
   ?o sc:name ?oName .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('pName', 'has parts of the class'), ('o', 'http://www.wikidata.org/entity/Q21170330'), ('oName', 'tennis qualification event')]
[('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P361'), ('oName', 'part of')]
[('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q16510064'), ('oName', 'sporting event')]
[('pName', 'sport'), ('o', 'http://www.wikidata.org/entity/Q847'), ('oName', 'tennis')]
[('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P641'), ('oName', 'sport')]
[('pName', 'part of'), ('o', 'http://www.wikidata.org/entity/Q47345468'), ('oName', 'tennis tournament edition')]
[('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P1346'), ('oName', 'winner')]
[('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P710'), ('oName', 'participant')]
[('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P2094'), ('oName', 'competition class')]


In [32]:
#Q1967745 type of tennis match
queryString = """
SELECT DISTINCT ?pName ?o ?oName
WHERE {
   wd:Q1967745 ?p ?o .
   ?p sc:name ?pName .
   ?o sc:name ?oName .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q22936940'), ('oName', 'competition class')]
[('pName', 'sport'), ('o', 'http://www.wikidata.org/entity/Q847'), ('oName', 'tennis')]
2


In [34]:
#Q1967745 type of tennis match
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   ?s ?p wd:Q1967745 .
   ?p sc:name ?pName .
   ?s sc:name ?sName .
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q18123880'), ('sName', 'tennis singles'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q18123885'), ('sName', 'tennis doubles'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q46190676'), ('sName', 'tennis event'), ('p', 'http://www.wikidata.org/prop/direct/P1552'), ('pName', 'has quality')]
[('s', 'http://www.wikidata.org/entity/Q50843310'), ('sName', 'tennis team event'), ('p', 'http://www.wikidata.org/prop/direct/P1552'), ('pName', 'has quality')]
[('s', 'http://www.wikidata.org/entity/Q108454403'), ('sName', "2022 US Open – Men's Singles"), ('p', 'http://www.wikidata.org/prop/direct/P1552'), ('pName', 'has quality')]
5


In [45]:
#Q46190676 tennis event
queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
    wd:Q1426 wdt:P2522 ?s .
   ?s ?p wd:Q46190676 .
   ?p sc:name ?pName .
   ?s sc:name ?sName .
   #FILTER REGEX(?sName , "Wimbledon*")
}
LIMIT 60
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q753300'), ('sName', "2012 Wimbledon Championships – Gentlemen's Singles"), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q782589'), ('sName', "2010 Australian Open – Men's Singles"), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q1074654'), ('sName', "2006 Wimbledon Championships – Men's Singles"), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q22956224'), ('sName', "2017 Australian Open – Men's Singles"), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q757054'), ('sName', "tennis at the 2008 Summer Olympics – men's doubles"), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q384929'), ('sName', "2005 Wi

In [46]:
queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName 
WHERE {
    wd:Q304072 ?p ?o .
    wd:Q248965 ?p ?o .
    ?p sc:name ?pName .
   ?o sc:name ?oName .
}
LIMIT 60
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner'), ('o', 'http://www.wikidata.org/entity/Q1426'), ('oName', 'Roger Federer')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('pName', 'competition class'), ('o', 'http://www.wikidata.org/entity/Q16893072'), ('oName', "men's singles")]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q46190676'), ('oName', 'tennis event')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('o', 'http://www.wikidata.org/entity/Q847'), ('oName', 'tennis')]
4


In [47]:
queryString = """
SELECT DISTINCT ?p ?pName ?s ?sName 
WHERE {
    ?s ?p wd:Q304072 .
    ?s ?p wd:Q248965 .
    ?p sc:name ?pName .
   ?s sc:name ?sName .
}
LIMIT 60
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('pName', 'victory'), ('s', 'http://www.wikidata.org/entity/Q1426'), ('sName', 'Roger Federer')]
1


In [53]:
queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName 
WHERE {
    wd:Q108454403 ?p ?o .
    ?p sc:name ?pName .
   ?o sc:name ?oName .
}
LIMIT 60
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('pName', 'has parts of the class'), ('o', 'http://www.wikidata.org/entity/Q21170330'), ('oName', 'tennis qualification event')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('o', 'http://www.wikidata.org/entity/Q847'), ('oName', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q46190676'), ('oName', 'tennis event')]
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('pName', 'has parts of the class'), ('o', 'http://www.wikidata.org/entity/Q47459169'), ('oName', 'tennis match')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('o', 'http://www.wikidata.org/entity/Q108454355'), ('oName', '2022 US Open')]
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('pName', 'has quality'), ('o', 'http://www.wikidata.org/entity/Q1967745'), ('oName', 'type of tennis match')]
[('p', 'http://www.wikidata.org

In [52]:
queryString = """
SELECT DISTINCT ?p ?pName ?s ?sName 
WHERE {
    ?s ?p wd:Q108454403 .
    ?p sc:name ?pName .
   ?s sc:name ?sName .
}
LIMIT 60
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('s', 'http://www.wikidata.org/entity/Q108454355'), ('sName', '2022 US Open')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by'), ('s', 'http://www.wikidata.org/entity/Q98811685'), ('sName', "2021 US Open – Men's Singles")]
2


In [54]:
#Q108454355  2022 US Open
queryString = """
SELECT DISTINCT ?p ?pName ?s ?sName 
WHERE {
    ?s ?p wd:Q108454355 .
    ?p sc:name ?pName .
   ?s sc:name ?sName .
}
LIMIT 60
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('s', 'http://www.wikidata.org/entity/Q108454406'), ('sName', '2022 US Open – Mixed Doubles')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pName', 'followed by'), ('s', 'http://www.wikidata.org/entity/Q98833450'), ('sName', '2021 US Open')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('s', 'http://www.wikidata.org/entity/Q108454402'), ('sName', "2022 US Open – Women's Singles")]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('s', 'http://www.wikidata.org/entity/Q108454404'), ('sName', "2022 US Open – Men's Doubles")]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('s', 'http://www.wikidata.org/entity/Q108454403'), ('sName', "2022 US Open – Men's Singles")]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('s', 'http://www.wikidata.org/entity/Q108454405'), ('sName', "2022 US Open – Women'

In [56]:
queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName 
WHERE {
    wd:Q123577 ?p ?o .
    ?p sc:name ?pName .
   ?o sc:name ?oName .
}
LIMIT 60
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pName', 'location'), ('o', 'http://www.wikidata.org/entity/Q1852081'), ('oName', 'USTA Billie Jean King National Tennis Center')]
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('pName', 'organizer'), ('o', 'http://www.wikidata.org/entity/Q948442'), ('oName', "Women's Tennis Association")]
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('pName', 'organizer'), ('o', 'http://www.wikidata.org/entity/Q223041'), ('oName', 'International Tennis Federation')]
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('pName', 'organizer'), ('o', 'http://www.wikidata.org/entity/Q1156323'), ('oName', 'United States Tennis Association')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q13219666'), ('oName', 'tennis tournament')]
[('p', 'http://www.wikidata.org/prop/direct/P765'), ('pName', 'surface played on'), ('o', 'http://www.wikidata.org/entity/Q1330871'), ('o

Final query for this task

In [59]:
# write your final query
#Q102113 Grand Slam of tennis
#P361 part of
queryString = """
SELECT DISTINCT ?grandslam ?grandslamName
WHERE {
    ?grandslam wdt:P361 wd:Q102113 .
    ?grandslam sc:name ?grandslamName .
}
LIMIT 60
"""

print("Results")
x=run_query(queryString)

Results
[('grandslam', 'http://www.wikidata.org/entity/Q43605'), ('grandslamName', 'French Open')]
[('grandslam', 'http://www.wikidata.org/entity/Q123577'), ('grandslamName', 'US Open')]
[('grandslam', 'http://www.wikidata.org/entity/Q41520'), ('grandslamName', 'Wimbledon Championships')]
[('grandslam', 'http://www.wikidata.org/entity/Q60874'), ('grandslamName', 'Australian Open')]
4


## Task 4
Find the three different type of possible surfaces in a tennis court (the result set must be a list of three couples surface IRI and label).

In [61]:
# write your queries
queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
    wd:Q60874 ?p ?o .
    ?p sc:name ?pName .
    ?o sc:name ?oName .
}
LIMIT 60
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('pName', 'organizer'), ('o', 'http://www.wikidata.org/entity/Q3494880'), ('oName', 'Tennis Australia')]
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('pName', 'organizer'), ('o', 'http://www.wikidata.org/entity/Q223041'), ('oName', 'International Tennis Federation')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pName', 'location'), ('o', 'http://www.wikidata.org/entity/Q2228393'), ('oName', 'Melbourne Park')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pName', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q5326540'), ('oName', 'Category:Australian Open (tennis)')]
[('p', 'http://www.wikidata.org/prop/direct/P4794'), ('pName', 'season starts'), ('o', 'http://www.wikidata.org/entity/Q108'), ('oName', 'January')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q13219666'), ('oName', 'tennis tournament')]
[('p', 'http:/

Final query for this task

In [318]:
# write your final query
#P765 urface played on
queryString = """
SELECT DISTINCT ?surface ?surfaceName
WHERE {
    ?s wdt:P765 ?surface .
    ?s sc:name ?sName .
    ?surface sc:name ?surfaceName .
}
LIMIT 3
"""

print("Results")
x=run_query(queryString)

Results
[('surface', 'http://www.wikidata.org/entity/Q47677214'), ('surfaceName', 'carpet')]
[('surface', 'http://www.wikidata.org/entity/Q133833'), ('surfaceName', 'gravel')]
[('surface', 'http://www.wikidata.org/entity/Q13731184'), ('surfaceName', 'artificial ice rink')]
3


## Task 5
Return the tournaments played in the 2018 Wimbledon Championships edition with the related winners (hint: men and women do not play together, there are two distinct tournament ) (the result set must be a list of elements with tournament IRI and label, and winner IRI and label).

In [66]:
# write your queries
#Q41520 Wimbledon Championships
queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
    wd:Q41520 ?p ?o .
    ?p sc:name ?pName .
    ?o sc:name ?oName .
}
LIMIT 60
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('pName', 'organizer'), ('o', 'http://www.wikidata.org/entity/Q815369'), ('oName', 'All England Lawn Tennis and Croquet Club')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pName', 'location'), ('o', 'http://www.wikidata.org/entity/Q815369'), ('oName', 'All England Lawn Tennis and Croquet Club')]
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('pName', 'organizer'), ('o', 'http://www.wikidata.org/entity/Q223041'), ('oName', 'International Tennis Federation')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q13219666'), ('oName', 'tennis tournament')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('o', 'http://www.wikidata.org/entity/Q145'), ('oName', 'United Kingdom')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q18608583'), ('oName', 'recurring spor

In [72]:
queryString = """
SELECT DISTINCT ?o ?oName #?s ?sName ?p ?pName 
WHERE {
    wd:Q1426 wdt:P2522 ?o .
   #?s ?p wd:Q46190676 .
   #?p sc:name ?pName .
   ?o sc:name ?oName .
   FILTER REGEX(?oName , "Wimbledon*")
}
LIMIT 60
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q753300'), ('oName', "2012 Wimbledon Championships – Gentlemen's Singles")]
[('o', 'http://www.wikidata.org/entity/Q1074654'), ('oName', "2006 Wimbledon Championships – Men's Singles")]
[('o', 'http://www.wikidata.org/entity/Q384929'), ('oName', "2005 Wimbledon Championships – Men's Singles")]
[('o', 'http://www.wikidata.org/entity/Q248965'), ('oName', "2004 Wimbledon Championships – Men's Singles")]
[('o', 'http://www.wikidata.org/entity/Q378627'), ('oName', "2003 Wimbledon Championships – Men's Singles")]
[('o', 'http://www.wikidata.org/entity/Q25344123'), ('oName', "2017 Wimbledon Championships – Men's Singles")]
[('o', 'http://www.wikidata.org/entity/Q751798'), ('oName', "2009 Wimbledon Championships – Men's Singles")]
[('o', 'http://www.wikidata.org/entity/Q912483'), ('oName', "2007 Wimbledon Championships – Men's Singles")]
8


In [76]:
#Q25344123 2017 Wimbledon Championships – Men's Singles
queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
    wd:Q25344123 ?p ?o .
   ?p sc:name ?pName .    
   ?o sc:name ?oName .
}
LIMIT 60
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('o', 'http://www.wikidata.org/entity/Q145'), ('oName', 'United Kingdom')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('o', 'http://www.wikidata.org/entity/Q847'), ('oName', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pName', 'winner'), ('o', 'http://www.wikidata.org/entity/Q1426'), ('oName', 'Roger Federer')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('pName', 'competition class'), ('o', 'http://www.wikidata.org/entity/Q16893072'), ('oName', "men's singles")]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q46190676'), ('oName', 'tennis event')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('o', 'http://www.wikidata.org/entity/Q25344121'), ('oName', '2017 Wimbledon Championships')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follow

In [77]:
#Q54961494 2018 Wimbledon Championships – Men's Singles
queryString = """
SELECT DISTINCT ?o ?oName
WHERE {
    wd:Q54961494 wdt:P1346 ?o .
   ?o sc:name ?oName .
}
LIMIT 60
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q5812'), ('oName', 'Novak Djokovic')]
1


In [81]:
#P1346 winner
queryString = """
SELECT DISTINCT ?sName ?o ?oName
WHERE {
    ?s wdt:P1346 ?o .
    ?s sc:name ?sName .
   ?o sc:name ?oName .
   FILTER REGEX(?sName , "2018 Wimbledon Championships*")
}
LIMIT 60
"""

print("Results")
x=run_query(queryString)

Results
[('sName', "2018 Wimbledon Championships – Women's Singles"), ('o', 'http://www.wikidata.org/entity/Q77178'), ('oName', 'Angelique Kerber')]
[('sName', '2018 Wimbledon Championships – Mixed Doubles'), ('o', 'http://www.wikidata.org/entity/Q2216368'), ('oName', 'Nicole Melichar-Martinez')]
[('sName', '2018 Wimbledon Championships – Mixed Doubles'), ('o', 'http://www.wikidata.org/entity/Q457476'), ('oName', 'Alexander Peya')]
[('sName', '2018 Wimbledon Championships – Wheelchair Quad Doubles'), ('o', 'http://www.wikidata.org/entity/Q13512827'), ('oName', 'Andrew Lapthorne')]
[('sName', "2018 Wimbledon Championships – Wheelchair Men's Doubles"), ('o', 'http://www.wikidata.org/entity/Q26951148'), ('oName', 'Alfie Hewett')]
[('sName', "2018 Wimbledon Championships – Men's Doubles"), ('o', 'http://www.wikidata.org/entity/Q53342'), ('oName', 'Mike Bryan')]
[('sName', "2018 Wimbledon Championships – Women's Doubles"), ('o', 'http://www.wikidata.org/entity/Q11052500'), ('oName', 'Barbor

Final query for this task

In [319]:
# write your final query
#P1346 winner
queryString = """
SELECT DISTINCT ?tournament ?tournamentName ?winner ?winnerName
WHERE {
    ?tournament wdt:P1346 ?winner .
    ?tournament sc:name ?tournamentName .
   ?winner sc:name ?winnerName .
   FILTER REGEX(?tournamentName , "2018 Wimbledon Championships*")
   #FILTER (REGEX(?tournamentName , "Men's Singles*") || REGEX(?tournamentName , "Women's Singles*"))
}
LIMIT 60
"""

print("Results")
x=run_query(queryString)

Results
[('tournament', 'http://www.wikidata.org/entity/Q30098268'), ('tournamentName', "2018 Wimbledon Championships – Women's Singles"), ('winner', 'http://www.wikidata.org/entity/Q77178'), ('winnerName', 'Angelique Kerber')]
[('tournament', 'http://www.wikidata.org/entity/Q32358401'), ('tournamentName', '2018 Wimbledon Championships – Mixed Doubles'), ('winner', 'http://www.wikidata.org/entity/Q2216368'), ('winnerName', 'Nicole Melichar-Martinez')]
[('tournament', 'http://www.wikidata.org/entity/Q32358401'), ('tournamentName', '2018 Wimbledon Championships – Mixed Doubles'), ('winner', 'http://www.wikidata.org/entity/Q457476'), ('winnerName', 'Alexander Peya')]
[('tournament', 'http://www.wikidata.org/entity/Q55639017'), ('tournamentName', '2018 Wimbledon Championships – Wheelchair Quad Doubles'), ('winner', 'http://www.wikidata.org/entity/Q13512827'), ('winnerName', 'Andrew Lapthorne')]
[('tournament', 'http://www.wikidata.org/entity/Q55455418'), ('tournamentName', "2018 Wimbledon 

## Task 6
Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

In [112]:
# write your queries
queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
    #?tennisplayer wdt:P106 wd:Q10833314 .
    #?tennisplayer ?p ?o .
    #wd:Q142 ?p ?o .
    wd:Q5 ?p ?o .
    ?p sc:name ?pName . 
    ?o sc:name ?oName . 
    FILTER REGEX(?oName, "date of birth")

}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('o', 'http://www.wikidata.org/entity/P569'), ('oName', 'date of birth')]
1


In [115]:
#P569 date of birth
queryString = """
SELECT DISTINCT ?pName 
WHERE {
    ?tennisplayer wdt:P106 wd:Q10833314 .
    ?tennisplayer ?p wd:P569 .
    ?p sc:name ?pName . 

}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [121]:
#P569 date of birth
queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
    wd:P569 ?p ?o .
    ?p sc:name ?pName . 
    ?o sc:name ?oName . 


}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3709'), ('pName', 'category for value different from Wikidata'), ('o', 'http://www.wikidata.org/entity/Q19332014'), ('oName', 'Category:Pages with P569 (date of birth) different from Wikidata')]
[('p', 'http://www.wikidata.org/prop/direct/P1659'), ('pName', 'see also'), ('o', 'http://www.wikidata.org/entity/P1317'), ('oName', 'floruit')]
[('p', 'http://www.wikidata.org/prop/direct/P1659'), ('pName', 'see also'), ('o', 'http://www.wikidata.org/entity/P3150'), ('oName', 'birthday')]
[('p', 'http://www.wikidata.org/prop/direct/P1855'), ('pName', 'Wikidata property example'), ('o', 'http://www.wikidata.org/entity/Q160432'), ('oName', 'Jude Law')]
[('p', 'http://www.wikidata.org/prop/direct/P1855'), ('pName', 'Wikidata property example'), ('o', 'http://www.wikidata.org/entity/Q935'), ('oName', 'Isaac Newton')]
[('p', 'http://www.wikidata.org/prop/direct/P1629'), ('pName', 'Wikidata item of this property'), ('o', 'http://www.wikidata.org/e

In [127]:
#P569 date of birth
#P1659 see also
#P3150 birthday
queryString = """
SELECT DISTINCT ?dateofbirth ?dateofbirthName ?birthdate ?birthdateName
WHERE {
    wd:Q5 wdt:P1963 ?dateofbirth .
    ?dateofbirth wdt:P1659 ?birthdate .

    ?dateofbirth sc:name ?dateofbirthName . 
    ?birthdate sc:name ?birthdateName . 


}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('dateofbirth', 'http://www.wikidata.org/entity/P937'), ('dateofbirthName', 'work location'), ('birthdate', 'http://www.wikidata.org/entity/P1317'), ('birthdateName', 'floruit')]
[('dateofbirth', 'http://www.wikidata.org/entity/P569'), ('dateofbirthName', 'date of birth'), ('birthdate', 'http://www.wikidata.org/entity/P1317'), ('birthdateName', 'floruit')]
[('dateofbirth', 'http://www.wikidata.org/entity/P570'), ('dateofbirthName', 'date of death'), ('birthdate', 'http://www.wikidata.org/entity/P1317'), ('birthdateName', 'floruit')]
[('dateofbirth', 'http://www.wikidata.org/entity/P3342'), ('dateofbirthName', 'significant person'), ('birthdate', 'http://www.wikidata.org/entity/P1327'), ('birthdateName', 'partner in business or sport')]
[('dateofbirth', 'http://www.wikidata.org/entity/P1559'), ('dateofbirthName', 'name in native language'), ('birthdate', 'http://www.wikidata.org/entity/P1477'), ('birthdateName', 'birth name')]
[('dateofbirth', 'http://www.wikidata.org/entity/P7

In [146]:
queryString = """
SELECT DISTINCT ?p1Name ?o1Name ?p2Name ?o2Name ?p3Name ?o3Name
WHERE {

    wd:Q11459 ?p1 ?o1 .
    ?o1 ?p2 ?o2.
    ?o2 ?p3 ?o3.
    
    ?p1 sc:name ?p1Name .
    ?p2 sc:name ?p2Name .
    ?p3 sc:name ?p3Name .
    ?o1 sc:name ?o1Name . 
    ?o2 sc:name ?o2Name . 
    ?o3 sc:name ?o3Name .
    
    FILTER (REGEX(?o1Name, "[Yy]ear*")||REGEX(?o2Name, "[Yy]ear*")||REGEX(?o3Name, "[Yy]ear*"))
    FILTER (REGEX(?p1Name, "[Yy]ear*")||REGEX(?p2Name, "[Yy]ear*")||REGEX(?p3Name, "[Yy]ear*"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p1Name', 'country for sport'), ('o1Name', 'United States of America'), ('p2Name', 'public holiday'), ('o2Name', "New Year's Day"), ('p3Name', 'day in year for periodic occurrence'), ('o3Name', 'January 1')]
[('p1Name', 'country of citizenship'), ('o1Name', 'United States of America'), ('p2Name', 'public holiday'), ('o2Name', "New Year's Day"), ('p3Name', 'day in year for periodic occurrence'), ('o3Name', 'January 1')]
2


In [148]:
queryString = """
SELECT DISTINCT ?p1Name ?o1Name ?p2Name ?o2Name ?p3Name ?o3Name
WHERE {

    wd:Q1426 ?p1 ?o1 .
    ?o1 ?p2 ?o2.
    ?o2 ?p3 ?o3.
    
    ?p1 sc:name ?p1Name .
    ?p2 sc:name ?p2Name .
    ?p3 sc:name ?p3Name .
    ?o1 sc:name ?o1Name . 
    ?o2 sc:name ?o2Name . 
    ?o3 sc:name ?o3Name .
    
    FILTER (REGEX(?o1Name, "[Yy]ear*")||REGEX(?o2Name, "[Yy]ear*")||REGEX(?o3Name, "[Yy]ear*"))
    FILTER (REGEX(?p1Name, "[Yy]ear*")||REGEX(?p2Name, "[Yy]ear*")||REGEX(?p3Name, "[Yy]ear*"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [173]:
queryString = """
SELECT DISTINCT ?p1Name ?o1Name ?p2Name ?o2Name ?p3Name ?o3Name
WHERE {

    wd:Q5 ?p1 ?o1 .
    ?o1 ?p2 ?o2.
    ?o2 ?p3 ?o3.
    
    ?p1 sc:name ?p1Name .
    ?p2 sc:name ?p2Name .
    ?p3 sc:name ?p3Name .
    ?o1 sc:name ?o1Name . 
    ?o2 sc:name ?o2Name . 
    ?o3 sc:name ?o3Name .
    
    FILTER (REGEX(?o1Name, "[Aa]ge*")||REGEX(?o2Name, "[Aa]ge*")||REGEX(?o3Name, "[Aa]ge*"))
    FILTER (REGEX(?p1Name, "[Aa]ge*")||REGEX(?p2Name, "[Aa]ge*")||REGEX(?p3Name, "[Aa]ge*"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p1Name', 'model item'), ('o1Name', 'Leonardo da Vinci'), ('p2Name', 'has works in the collection'), ('o2Name', 'Czartoryski Museum'), ('p3Name', 'heritage designation'), ('o3Name', 'object on the Polish cultural heritage register')]
[('p1Name', 'model item'), ('o1Name', 'Leonardo da Vinci'), ('p2Name', 'has works in the collection'), ('o2Name', 'Gabinetto dei disegni e delle stampe'), ('p3Name', 'heritage designation'), ('o3Name', 'Italian national heritage')]
[('p1Name', 'has quality'), ('o1Name', 'personal name'), ('p2Name', 'Wikidata property'), ('o2Name', 'name in native language'), ('p3Name', 'property usage tracking category'), ('o3Name', 'Category:Pages using Wikidata property P1559')]
[('p1Name', 'has quality'), ('o1Name', 'personal name'), ('p2Name', 'properties for this type'), ('o2Name', 'name in native language'), ('p3Name', 'property usage tracking category'), ('o3Name', 'Category:Pages using Wikidata property P1559')]
[('p1Name', 'different from'), ('o1Name', '

In [178]:
queryString = """
SELECT DISTINCT ?p1Name ?o1Name ?p2Name ?o2Name ?p3Name ?o3Name  #datatype(?o1Name)
WHERE {

    wd:Q5 ?p1 ?o1 .
    ?o1 ?p2 ?o2.
    ?o2 ?p3 ?o3.
    
    ?p1 sc:name ?p1Name .
    ?p2 sc:name ?p2Name .
    ?p3 sc:name ?p3Name .
    ?o1 sc:name ?o1Name . 
    ?o2 sc:name ?o2Name . 
    ?o3 sc:name ?o3Name .
    
    FILTER ( datatype(?o1Name) = xsd:string  || datatype(?o2Name) = xsd:integer  || datatype(?o3Name) = xsd:integer  )
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)


Results
Empty


In [180]:
queryString = """
SELECT DISTINCT ?tennisplayerName ?birthdate
WHERE {
    ?tennisplayer wdt:P106 wd:Q10833314 ;
            sc:name ?tennisplayerName .
    ?tennisplayer wdt:P569 ?birthdate .
   FILTER (datatype(?birthdate) = xsd:dateTime)
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('tennisplayerName', 'Jaroslav Navrátil'), ('birthdate', '1957-07-24T00:00:00Z')]
[('tennisplayerName', 'Anna Karolína Schmiedlová'), ('birthdate', '1994-09-13T00:00:00Z')]
[('tennisplayerName', 'Wesley Koolhof'), ('birthdate', '1989-04-17T00:00:00Z')]
[('tennisplayerName', 'Pierre Barthès'), ('birthdate', '1941-09-13T00:00:00Z')]
[('tennisplayerName', 'Sue Partridge'), ('birthdate', '1930-01-01T00:00:00Z')]
[('tennisplayerName', 'Paul Oosterbaan'), ('birthdate', '1995-07-08T00:00:00Z')]
[('tennisplayerName', 'Rika Fujiwara'), ('birthdate', '1981-09-19T00:00:00Z')]
[('tennisplayerName', 'Margarita Gasparyan'), ('birthdate', '1994-09-01T00:00:00Z')]
[('tennisplayerName', 'Philippe Chatrier'), ('birthdate', '1928-02-02T00:00:00Z')]
[('tennisplayerName', 'James Bayley'), ('birthdate', '1899-11-04T00:00:00Z')]
[('tennisplayerName', 'Samuel T. Chase'), ('birthdate', '1868-09-14T00:00:00Z')]
[('tennisplayerName', 'Agnete Goldschmidt'), ('birthdate', '1899-08-04T00:00:00Z')]
[('tenni

In [182]:
queryString = """
SELECT DISTINCT ?tennisplayerName ?birthdate
WHERE {
    ?tennisplayer wdt:P106 wd:Q10833314 ;
            sc:name ?tennisplayerName ;
            wdt:P569 ?birthdate .
   FILTER (datatype(?birthdate) = xsd:dateTime)
   FILTER (?birthdate >= '1975-01-01T00:00:00Z'^^xsd:dateTime)
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('tennisplayerName', 'Anna Karolína Schmiedlová'), ('birthdate', '1994-09-13T00:00:00Z')]
[('tennisplayerName', 'Wesley Koolhof'), ('birthdate', '1989-04-17T00:00:00Z')]
[('tennisplayerName', 'Paul Oosterbaan'), ('birthdate', '1995-07-08T00:00:00Z')]
[('tennisplayerName', 'Rika Fujiwara'), ('birthdate', '1981-09-19T00:00:00Z')]
[('tennisplayerName', 'Margarita Gasparyan'), ('birthdate', '1994-09-01T00:00:00Z')]
[('tennisplayerName', 'Lee So-ra'), ('birthdate', '1994-07-22T00:00:00Z')]
[('tennisplayerName', 'Radu Albot'), ('birthdate', '1989-11-11T00:00:00Z')]
[('tennisplayerName', 'Markéta Vondroušová'), ('birthdate', '1999-06-28T00:00:00Z')]
[('tennisplayerName', 'Djordje Djokovic'), ('birthdate', '1995-07-07T00:00:00Z')]
[('tennisplayerName', 'Sviatlana Pirazhenka'), ('birthdate', '1992-09-12T00:00:00Z')]
[('tennisplayerName', 'Marcelo Zormann'), ('birthdate', '1996-06-10T00:00:00Z')]
[('tennisplayerName', 'Sanaz Marand'), ('birthdate', '1988-06-21T00:00:00Z')]
[('tennisplay

In [205]:
queryString = """
SELECT ?tennisplayerName ?birthdate ?titleName
WHERE {
    ?tennisplayer wdt:P106 wd:Q10833314 ;
            sc:name ?tennisplayerName ;
            wdt:P569 ?birthdate ;
             wdt:P2522 ?title .
    ?title sc:name ?titleName .
   FILTER (datatype(?birthdate) = xsd:dateTime)
   FILTER (?birthdate >= '1975-01-01T00:00:00Z'^^xsd:dateTime)
   FILTER (REGEX(?titleName, 'French Open*')|| REGEX(?titleName, 'Australian Open*') || REGEX(?titleName, 'Wimbledon Championships*') || REGEX(?titleName, 'US Open*') )
}
LIMIT 150
"""

print("Results")
x=run_query(queryString)

Results
[('tennisplayerName', 'Victoria Azarenka'), ('birthdate', '1989-07-31T00:00:00Z'), ('titleName', "2012 Australian Open – Women's Singles")]
[('tennisplayerName', 'Henri Kontinen'), ('birthdate', '1990-06-19T00:00:00Z'), ('titleName', 'Australian Open')]
[('tennisplayerName', 'Sania Mirza'), ('birthdate', '1986-11-15T00:00:00Z'), ('titleName', '2012 French Open – Mixed Doubles')]
[('tennisplayerName', 'Kim Clijsters'), ('birthdate', '1983-06-08T00:00:00Z'), ('titleName', 'French Open')]
[('tennisplayerName', 'Garbiñe Muguruza'), ('birthdate', '1993-10-08T00:00:00Z'), ('titleName', "2017 Wimbledon Championships – Women's Singles")]
[('tennisplayerName', 'Garbiñe Muguruza'), ('birthdate', '1993-10-08T00:00:00Z'), ('titleName', "2016 French Open – Women's Singles")]
[('tennisplayerName', 'Serena Williams'), ('birthdate', '1981-09-26T00:00:00Z'), ('titleName', 'French Open')]
[('tennisplayerName', 'Anna Kournikova'), ('birthdate', '1981-06-07T00:00:00Z'), ('titleName', "1999 Austral

In [209]:
queryString = """
SELECT ?tennisplayer ?tennisplayerName ?birthdate ?titleName
WHERE {
    ?tennisplayer wdt:P106 wd:Q10833314 ;
            sc:name ?tennisplayerName ;
            wdt:P569 ?birthdate ;
             wdt:P2522 ?title .
    ?title sc:name ?titleName .
   FILTER (datatype(?birthdate) = xsd:dateTime)
   FILTER (?birthdate >= '1975-01-01T00:00:00Z'^^xsd:dateTime)
   FILTER (REGEX(?titleName, 'French*')|| REGEX(?titleName, 'Australian*') || REGEX(?titleName, 'Wimbledon *') || REGEX(?titleName, 'US*'))
}
LIMIT 150
"""

print("Results")
x=run_query(queryString)

Results
[('tennisplayer', 'http://www.wikidata.org/entity/Q10118'), ('tennisplayerName', 'Victoria Azarenka'), ('birthdate', '1989-07-31T00:00:00Z'), ('titleName', "2012 Australian Open – Women's Singles")]
[('tennisplayer', 'http://www.wikidata.org/entity/Q13538'), ('tennisplayerName', 'Henri Kontinen'), ('birthdate', '1990-06-19T00:00:00Z'), ('titleName', 'Australian Open')]
[('tennisplayer', 'http://www.wikidata.org/entity/Q132147'), ('tennisplayerName', 'Sania Mirza'), ('birthdate', '1986-11-15T00:00:00Z'), ('titleName', '2012 French Open – Mixed Doubles')]
[('tennisplayer', 'http://www.wikidata.org/entity/Q11685'), ('tennisplayerName', 'Kim Clijsters'), ('birthdate', '1983-06-08T00:00:00Z'), ('titleName', 'French Open')]
[('tennisplayer', 'http://www.wikidata.org/entity/Q34276'), ('tennisplayerName', 'Garbiñe Muguruza'), ('birthdate', '1993-10-08T00:00:00Z'), ('titleName', "2017 Wimbledon Championships – Women's Singles")]
[('tennisplayer', 'http://www.wikidata.org/entity/Q34276')

In [212]:
# Q10132 Rafael Nadal won titles
queryString = """
SELECT ?titleName
WHERE {

    wd:Q10132 wdt:P569 ?birthdate ;
             wdt:P2522 ?title .
    ?title sc:name ?titleName .
   #FILTER (REGEX(?titleName, 'French*')|| REGEX(?titleName, 'Australian*') || REGEX(?titleName, 'Wimbledon *') || REGEX(?titleName, 'US*'))
}
LIMIT 150
"""

print("Results")
x=run_query(queryString)

Results
[('titleName', 'French Open')]
1


In [236]:
# Q10132 Rafael Nadal 
queryString = """
SELECT ?pName ?oName
WHERE {
    wd:Q10132 wdt:P569 ?birthdate ;
             ?p ?o .
    ?p sc:name ?pName .
    ?o sc:name ?oName .
}
LIMIT 150
"""

print("Results")
x=run_query(queryString)

Results
[('pName', 'relative'), ('oName', 'Rafael Nadal Nadal')]
[('pName', 'relative'), ('oName', 'Toni Nadal')]
[('pName', 'relative'), ('oName', 'Miguel Ángel Nadal')]
[('pName', 'occupation'), ('oName', 'tennis player')]
[('pName', 'occupation'), ('oName', 'model')]
[('pName', 'described by source'), ('oName', 'Lentapedia')]
[('pName', 'described by source'), ('oName', 'Obalky knih.cz')]
[('pName', 'participant in'), ('oName', '2009 Davis Cup')]
[('pName', 'participant in'), ('oName', '2013 Davis Cup')]
[('pName', 'participant in'), ('oName', '2015 Davis Cup')]
[('pName', 'participant in'), ('oName', 'tennis at the 2004 Summer Olympics')]
[('pName', 'participant in'), ('oName', '2018 Davis Cup')]
[('pName', 'participant in'), ('oName', '2011 Davis Cup')]
[('pName', 'participant in'), ('oName', 'tennis at the 2008 Summer Olympics')]
[('pName', 'participant in'), ('oName', 'tennis at the 2016 Summer Olympics')]
[('pName', 'participant in'), ('oName', "tennis at the 2016 Summer Olympi

In [339]:
queryString = """
SELECT ?tennisplayer ?tennisplayerName (COUNT(?titleName) AS ?numtitle)
WHERE 
{
    ?tennisplayer wdt:P106 wd:Q10833314 ;
            sc:name ?tennisplayerName ;
            wdt:P569 ?birthdate ;
            ?p ?title .
             #wdt:P2522 ?title .
    ?title sc:name ?titleName .
    ?p sc:name ?pName .
   FILTER (datatype(?birthdate) = xsd:dateTime)
   FILTER (?birthdate >= '1975-01-01T00:00:00Z'^^xsd:dateTime)
   FILTER REGEX(?pName, 'victory')
   FILTER (REGEX(?titleName, 'French Open*')|| REGEX(?titleName, 'Australian Open*') || REGEX(?titleName, 'Wimbledon Championships*') || REGEX(?titleName, 'US Open*') )
}
GROUP BY ?tennisplayer ?tennisplayerName #?titleName ?pName 
ORDER BY DESC (?numtitle)
LIMIT 170
"""

print("Results")
x=run_query(queryString)

Results
[('tennisplayer', 'http://www.wikidata.org/entity/Q1426'), ('tennisplayerName', 'Roger Federer'), ('numtitle', '20')]
[('tennisplayer', 'http://www.wikidata.org/entity/Q17466583'), ('tennisplayerName', 'Naomi Osaka'), ('numtitle', '4')]
[('tennisplayer', 'http://www.wikidata.org/entity/Q26840238'), ('tennisplayerName', 'Iga Świątek'), ('numtitle', '3')]
[('tennisplayer', 'http://www.wikidata.org/entity/Q190723'), ('tennisplayerName', 'Gustavo Kuerten'), ('numtitle', '3')]
[('tennisplayer', 'http://www.wikidata.org/entity/Q10125'), ('tennisplayerName', 'Andy Murray'), ('numtitle', '2')]
[('tennisplayer', 'http://www.wikidata.org/entity/Q230156'), ('tennisplayerName', 'Simona Halep'), ('numtitle', '2')]
[('tennisplayer', 'http://www.wikidata.org/entity/Q34276'), ('tennisplayerName', 'Garbiñe Muguruza'), ('numtitle', '2')]
[('tennisplayer', 'http://www.wikidata.org/entity/Q10118'), ('tennisplayerName', 'Victoria Azarenka'), ('numtitle', '1')]
[('tennisplayer', 'http://www.wikidata

Final query for this task

In [336]:
# write your final query
queryString = """
SELECT ?tennisplayer ?tennisplayerName (COUNT(?titleName) AS ?numtitle)
WHERE 
{
    ?tennisplayer wdt:P106 wd:Q10833314 ;
            sc:name ?tennisplayerName ;
            wdt:P569 ?birthdate ;
             wdt:P2522 ?title .
    ?title sc:name ?titleName .
   FILTER (datatype(?birthdate) = xsd:dateTime)
   FILTER (?birthdate >= '1975-01-01T00:00:00Z'^^xsd:dateTime)
   FILTER (REGEX(?titleName, 'French Open*')|| REGEX(?titleName, 'Australian Open*') || REGEX(?titleName, 'Wimbledon Championships*') || REGEX(?titleName, 'US Open*') )
}
GROUP BY ?tennisplayer ?tennisplayerName
ORDER BY DESC (?numtitle)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('tennisplayer', 'http://www.wikidata.org/entity/Q1426'), ('tennisplayerName', 'Roger Federer'), ('numtitle', '20')]
[('tennisplayer', 'http://www.wikidata.org/entity/Q17466583'), ('tennisplayerName', 'Naomi Osaka'), ('numtitle', '4')]
[('tennisplayer', 'http://www.wikidata.org/entity/Q26840238'), ('tennisplayerName', 'Iga Świątek'), ('numtitle', '3')]
[('tennisplayer', 'http://www.wikidata.org/entity/Q190723'), ('tennisplayerName', 'Gustavo Kuerten'), ('numtitle', '3')]
[('tennisplayer', 'http://www.wikidata.org/entity/Q10125'), ('tennisplayerName', 'Andy Murray'), ('numtitle', '2')]
[('tennisplayer', 'http://www.wikidata.org/entity/Q230156'), ('tennisplayerName', 'Simona Halep'), ('numtitle', '2')]
[('tennisplayer', 'http://www.wikidata.org/entity/Q34276'), ('tennisplayerName', 'Garbiñe Muguruza'), ('numtitle', '2')]
[('tennisplayer', 'http://www.wikidata.org/entity/Q10118'), ('tennisplayerName', 'Victoria Azarenka'), ('numtitle', '1')]
[('tennisplayer', 'http://www.wikidata

## Task 7
Consider the country for sport property of the tennis player (NO Country of citizenship). Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

In [234]:
# write your queries
# Q10132 Rafael Nadal
queryString = """
SELECT ?p ?pName ?oName
WHERE {
    wd:Q10132 wdt:P569 ?birthdate ;
             ?p ?o .
    ?p sc:name ?pName .
    ?o sc:name ?oName .
}
LIMIT 150
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1038'), ('pName', 'relative'), ('oName', 'Rafael Nadal Nadal')]
[('p', 'http://www.wikidata.org/prop/direct/P1038'), ('pName', 'relative'), ('oName', 'Toni Nadal')]
[('p', 'http://www.wikidata.org/prop/direct/P1038'), ('pName', 'relative'), ('oName', 'Miguel Ángel Nadal')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('oName', 'tennis player')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('oName', 'model')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('oName', 'Lentapedia')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('oName', 'Obalky knih.cz')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pName', 'participant in'), ('oName', '2009 Davis Cup')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pName', 'participant in'), ('oName', '2013 Davis Cup')]
[('p', 'http://www.wi

In [229]:
#P1532 country for sport
queryString = """
SELECT ?country ?countryName  ?tennisplayerName ?titleName 
WHERE {
    ?tennisplayer wdt:P106 wd:Q10833314 ;
            sc:name ?tennisplayerName ;
            wdt:P2522 ?title .
    ?title sc:name ?titleName .
    ?tennisplayer wdt:P1532 ?country .
     ?country sc:name ?countryName .
   FILTER (REGEX(?titleName, 'French Open*')|| REGEX(?titleName, 'Australian Open*') || REGEX(?titleName, 'Wimbledon Championships*') || REGEX(?titleName, 'US Open*') )

}
#GROUP BY ?country ?countryName 

LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryName', 'United Kingdom'), ('tennisplayerName', 'Virginia Wade'), ('titleName', "1977 Wimbledon Championships – Women's Singles")]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryName', 'United Kingdom'), ('tennisplayerName', 'Ann Haydon-Jones'), ('titleName', "1969 Wimbledon Championships – Women's Singles")]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany'), ('tennisplayerName', 'Steffi Graf'), ('titleName', "1987 French Open – Women's Singles")]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany'), ('tennisplayerName', 'Steffi Graf'), ('titleName', "1988 French Open – Women's Singles")]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany'), ('tennisplayerName', 'Steffi Graf'), ('titleName', "1999 French Open – Women's Singles")]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany'), ('tennispla

Final query for this task

In [258]:
# write your final query
queryString = """
SELECT DISTINCT ?country ?countryName
WHERE {
    ?tennisplayer wdt:P106 wd:Q10833314 ;
            sc:name ?tennisplayerName ;
            wdt:P2522 ?title .
    ?title sc:name ?titleName .
    ?tennisplayer wdt:P1532 ?country .
     ?country sc:name ?countryName .
   FILTER (REGEX(?titleName, 'French Open*')|| REGEX(?titleName, 'Australian Open*') || REGEX(?titleName, 'Wimbledon Championships*') || REGEX(?titleName, 'US Open*') )

}

LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryName', 'United Kingdom')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany')]
[('country', 'http://www.wikidata.org/entity/Q184'), ('countryName', 'Belarus')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('countryName', 'Spain')]
[('country', 'http://www.wikidata.org/entity/Q30'), ('countryName', 'United States of America')]
[('country', 'http://www.wikidata.org/entity/Q31'), ('countryName', 'Belgium')]
[('country', 'http://www.wikidata.org/entity/Q33'), ('countryName', 'Finland')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryName', 'Italy')]
[('country', 'http://www.wikidata.org/entity/Q668'), ('countryName', 'India')]
[('country', 'http://www.wikidata.org/entity/Q159'), ('countryName', 'Russia')]
[('country', 'http://www.wikidata.org/entity/Q218'), ('countryName', 'Romania')]
[('country', 'http://www.wikidata.org/entity/Q34'), ('countryName', 'Sweden')]
[('country

## Task 8
Consider the country for sport property of the tennis player (NO Country of citizenship). Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

In [238]:
# write your queries
queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   wd:Q1426 ?p ?o .
   ?p sc:name ?pName .
   ?o sc:name ?oName .
   FILTER REGEX(?oName, 'ATP*') 
}
LIMIT 120
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('pName', 'victory'), ('o', 'http://www.wikidata.org/entity/Q300024'), ('oName', '2011 ATP World Tour Finals')]
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('pName', 'victory'), ('o', 'http://www.wikidata.org/entity/Q300022'), ('oName', '2010 ATP World Tour Finals')]
2


In [260]:
# write your queries
queryString = """
SELECT DISTINCT ?tennisplayer ?tennisplayerName ?o ?oName
WHERE 
{    
    ?tennisplayer wdt:P106 wd:Q10833314 ;
            sc:name ?tennisplayerName ;
                 ?p ?o .
   ?o sc:name ?oName .
   #FILTER (datatype(?o) = xsd:int)
}
LIMIT 250
"""

print("Results")
x=run_query(queryString)

Results
[('tennisplayer', 'http://www.wikidata.org/entity/Q1683632'), ('tennisplayerName', 'Jaroslav Navrátil'), ('o', 'http://www.wikidata.org/entity/Q13639102'), ('oName', 'Jaroslav Navrátil')]
[('tennisplayer', 'http://www.wikidata.org/entity/Q589611'), ('tennisplayerName', 'Philippe Chatrier'), ('o', 'http://www.wikidata.org/entity/Q15849678'), ('oName', 'Sue Partridge')]
[('tennisplayer', 'http://www.wikidata.org/entity/Q15197746'), ('tennisplayerName', 'Wesley Koolhof'), ('o', 'http://www.wikidata.org/entity/Q15906750'), ('oName', 'Dean Koolhof')]
[('tennisplayer', 'http://www.wikidata.org/entity/Q15197746'), ('tennisplayerName', 'Wesley Koolhof'), ('o', 'http://www.wikidata.org/entity/Q2979498'), ('oName', 'Jurrie Koolhof')]
[('tennisplayer', 'http://www.wikidata.org/entity/Q15880574'), ('tennisplayerName', 'Paul Oosterbaan'), ('o', 'http://www.wikidata.org/entity/Q36873366'), ('oName', 'Oosterbaan')]
[('tennisplayer', 'http://www.wikidata.org/entity/Q15197746'), ('tennisplayerN

In [262]:
# Q5812 Novak Djokovic
queryString = """
SELECT DISTINCT ?tennisplayer ?tennisplayerName ?pName ?o ?oName
WHERE 
{    
    wd:Q5812 wdt:P106 wd:Q10833314 ;
            sc:name ?tennisplayerName ;
                 ?p ?o .
     ?p sc:name ?pName .            
   ?o sc:name ?oName .
   #FILTER (datatype(?o) = xsd:int)
}
LIMIT 250
"""

print("Results")
x=run_query(queryString)

Results
[('tennisplayerName', 'Novak Djokovic'), ('pName', 'native language'), ('o', 'http://www.wikidata.org/entity/Q9299'), ('oName', 'Serbian')]
[('tennisplayerName', 'Novak Djokovic'), ('pName', 'medical condition'), ('o', 'http://www.wikidata.org/entity/Q11088'), ('oName', 'coeliac disease')]
[('tennisplayerName', 'Novak Djokovic'), ('pName', 'medical condition'), ('o', 'http://www.wikidata.org/entity/Q84263196'), ('oName', 'COVID-19')]
[('tennisplayerName', 'Novak Djokovic'), ('pName', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q10833314'), ('oName', 'tennis player')]
[('tennisplayerName', 'Novak Djokovic'), ('pName', 'employer'), ('o', 'http://www.wikidata.org/entity/Q740308'), ('oName', 'UNICEF')]
[('tennisplayerName', 'Novak Djokovic'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q67311526'), ('oName', 'Obalky knih.cz')]
[('tennisplayerName', 'Novak Djokovic'), ('pName', 'participant in'), ('o', 'http://www.wikidata.org/entity/Q65242348'),

In [268]:
# Q5812 Novak Djokovic
queryString = """
SELECT DISTINCT ?tennisplayer ?tennisplayerName ?sName ?pName
WHERE 
{    
    wd:Q5812 wdt:P106 wd:Q10833314 ;
            sc:name ?tennisplayerName .
       ?s ?p wd:Q5812 .
     ?p sc:name ?pName .            
   ?s sc:name ?sName .
   #FILTER (datatype(?o) = xsd:int)
}
LIMIT 250
"""

print("Results")
x=run_query(queryString)

Results
[('tennisplayerName', 'Novak Djokovic'), ('sName', 'Template:Novak Djokovic'), ('pName', 'template has topic')]
[('tennisplayerName', 'Novak Djokovic'), ('sName', '2012 Novak Djokovic tennis season'), ('pName', 'facet of')]
[('tennisplayerName', 'Novak Djokovic'), ('sName', 'list of career achievements by Novak Djokovic'), ('pName', 'facet of')]
[('tennisplayerName', 'Novak Djokovic'), ('sName', '2010 Novak Djokovic tennis season'), ('pName', 'facet of')]
[('tennisplayerName', 'Novak Djokovic'), ('sName', 'Djokovic–Murray rivalry'), ('pName', 'facet of')]
[('tennisplayerName', 'Novak Djokovic'), ('sName', '2016 Novak Djokovic tennis season'), ('pName', 'facet of')]
[('tennisplayerName', 'Novak Djokovic'), ('sName', '2017 Novak Djokovic tennis season'), ('pName', 'facet of')]
[('tennisplayerName', 'Novak Djokovic'), ('sName', '2015 Novak Djokovic tennis season'), ('pName', 'facet of')]
[('tennisplayerName', 'Novak Djokovic'), ('sName', '2013 Novak Djokovic tennis season'), ('pNa

In [269]:
# Q5812 Novak Djokovic
# Q1426 Roger
queryString = """
SELECT DISTINCT ?tennisplayer ?tennisplayerName ?pName ?o ?oName
WHERE 
{    wd:Q1426 ?p ?o .
    wd:Q5812 wdt:P106 wd:Q10833314 ;
            sc:name ?tennisplayerName ;
                 ?p ?o .
     ?p sc:name ?pName .            
   ?o sc:name ?oName .
}
LIMIT 250
"""

print("Results")
x=run_query(queryString)

Results
[('tennisplayerName', 'Novak Djokovic'), ('pName', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q10833314'), ('oName', 'tennis player')]
[('tennisplayerName', 'Novak Djokovic'), ('pName', 'languages spoken, written or signed'), ('o', 'http://www.wikidata.org/entity/Q150'), ('oName', 'French')]
[('tennisplayerName', 'Novak Djokovic'), ('pName', 'languages spoken, written or signed'), ('o', 'http://www.wikidata.org/entity/Q1860'), ('oName', 'English')]
[('tennisplayerName', 'Novak Djokovic'), ('pName', 'languages spoken, written or signed'), ('o', 'http://www.wikidata.org/entity/Q188'), ('oName', 'German')]
[('tennisplayerName', 'Novak Djokovic'), ('pName', 'position held'), ('o', 'http://www.wikidata.org/entity/Q19969187'), ('oName', 'UNICEF Goodwill Ambassador')]
[('tennisplayerName', 'Novak Djokovic'), ('pName', 'handedness'), ('o', 'http://www.wikidata.org/entity/Q3039938'), ('oName', 'right-handedness')]
[('tennisplayerName', 'Novak Djokovic'), ('pName', 'instance of

In [280]:
# Q17466583 Naomi Osaka
queryString = """
SELECT DISTINCT ?tennisplayer ?tennisplayerName ?p ?pName ?o ?oName
WHERE 
{
    wd:Q17466583 wdt:P106 wd:Q10833314 ;
            sc:name ?tennisplayerName ;
                 ?p ?o .
     ?p sc:name ?pName .            
   ?o sc:name ?oName .
}
LIMIT 250
"""

print("Results")
x=run_query(queryString)

Results
[('tennisplayerName', 'Naomi Osaka'), ('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q10833314'), ('oName', 'tennis player')]
[('tennisplayerName', 'Naomi Osaka'), ('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pName', 'participant in'), ('o', 'http://www.wikidata.org/entity/Q30103106'), ('oName', '2018 Fed Cup')]
[('tennisplayerName', 'Naomi Osaka'), ('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pName', 'participant in'), ('o', 'http://www.wikidata.org/entity/Q181278'), ('oName', '2020 Summer Olympics')]
[('tennisplayerName', 'Naomi Osaka'), ('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pName', 'participant in'), ('o', 'http://www.wikidata.org/entity/Q65242336'), ('oName', "tennis at the 2020 Summer Olympics – women's singles")]
[('tennisplayerName', 'Naomi Osaka'), ('p', 'http://www.wikidata.org/prop/direct/P1412'), ('pName', 'languages spoken, written or signed'), ('o', 'http://www.wiki

In [279]:
queryString = """
SELECT ?tennisplayer ?tennisplayerName ?pName ?o ?oName
WHERE 
{
    ?tennisplayer wdt:P106 wd:Q10833314 ;
            sc:name ?tennisplayerName ;
                 ?p ?o .
     ?p sc:name ?pName .            
       ?o sc:name ?oName .
   
    #FILTER (datatype(?oName) = xsd:string)
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('tennisplayer', 'http://www.wikidata.org/entity/Q207352'), ('tennisplayerName', 'Agnieszka Radwańska'), ('pName', 'native language'), ('o', 'http://www.wikidata.org/entity/Q809'), ('oName', 'Polish')]
[('tennisplayer', 'http://www.wikidata.org/entity/Q16224631'), ('tennisplayerName', 'Lee So-ra'), ('pName', 'native language'), ('o', 'http://www.wikidata.org/entity/Q9176'), ('oName', 'Korean')]
[('tennisplayer', 'http://www.wikidata.org/entity/Q207352'), ('tennisplayerName', 'Agnieszka Radwańska'), ('pName', 'relative'), ('o', 'http://www.wikidata.org/entity/Q14616725'), ('oName', 'Władysław Radwański')]
[('tennisplayer', 'http://www.wikidata.org/entity/Q18918709'), ('tennisplayerName', 'Markéta Vondroušová'), ('pName', 'medical condition'), ('o', 'http://www.wikidata.org/entity/Q84263196'), ('oName', 'COVID-19')]
[('tennisplayer', 'http://www.wikidata.org/entity/Q14905846'), ('tennisplayerName', 'Borna Ćorić'), ('pName', 'medical condition'), ('o', 'http://www.wikidata.org/en

In [281]:
#P1532 country for sport
#Q847 tennis
queryString = """
SELECT ?p ?pName ?o ?oName
WHERE 
{
    wd:Q847 ?p ?o.
    ?p sc:name ?pName .
    ?o sc:name ?oName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2283'), ('pName', 'uses'), ('o', 'http://www.wikidata.org/entity/Q153362'), ('oName', 'tennis racquet')]
[('p', 'http://www.wikidata.org/prop/direct/P6104'), ('pName', 'maintained by WikiProject'), ('o', 'http://www.wikidata.org/entity/Q5530169'), ('oName', 'WikiProject Tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P797'), ('pName', 'authority'), ('o', 'http://www.wikidata.org/entity/Q223041'), ('oName', 'International Tennis Federation')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q106727050'), ('oName', 'Metropolitan Museum of Art Tagging Vocabulary')]
[('p', 'http://www.wikidata.org/prop/direct/P3095'), ('pName', 'practiced by'), ('o', 'http://www.wikidata.org/entity/Q10833314'), ('oName', 'tennis player')]
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('pName', 'country of origin'), ('o', 'http://www.wikidata.org/entity/Q21'), ('oName'

In [285]:
# Q48673045 professional tennis player
queryString = """
SELECT ?p ?pName ?o ?oName
WHERE 
{
    wd:Q48673045 ?p ?o.
    ?p sc:name ?pName .
    ?o sc:name ?oName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q10833314'), ('oName', 'tennis player')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q28640'), ('oName', 'profession')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('o', 'http://www.wikidata.org/entity/Q847'), ('oName', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P425'), ('pName', 'field of this occupation'), ('o', 'http://www.wikidata.org/entity/Q847'), ('oName', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P461'), ('pName', 'opposite of'), ('o', 'http://www.wikidata.org/entity/Q48995125'), ('oName', 'amateur tennis player')]
5


In [286]:
# Q223041 International Tennis Federation
queryString = """
SELECT ?p ?pName ?o ?oName
WHERE 
{
    wd:Q223041 ?p ?o.
    ?p sc:name ?pName .
    ?o sc:name ?oName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q11422536'), ('oName', 'international sport governing body')]
[('p', 'http://www.wikidata.org/prop/direct/P463'), ('pName', 'member of'), ('o', 'http://www.wikidata.org/entity/Q1067053'), ('oName', 'Global Association of International Sports Federations')]
[('p', 'http://www.wikidata.org/prop/direct/P2541'), ('pName', 'operating area'), ('o', 'http://www.wikidata.org/entity/Q2'), ('oName', 'Earth')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('pName', 'headquarters location'), ('o', 'http://www.wikidata.org/entity/Q84'), ('oName', 'London')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('o', 'http://www.wikidata.org/entity/Q847'), ('oName', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q98426739'), ('oName', 'tennis association')]
[('p', 'http://www.w

In [287]:
# Q98426739 tennis association
queryString = """
SELECT ?p ?pName ?o ?oName
WHERE 
{
    wd:Q98426739 ?p ?o.
    ?p sc:name ?pName .
    ?o sc:name ?oName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q2485448'), ('oName', 'sports governing body')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('o', 'http://www.wikidata.org/entity/Q847'), ('oName', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pName', "topic's main category"), ('o', 'http://www.wikidata.org/entity/Q9156493'), ('oName', 'Category:Tennis organizations')]
3


In [288]:
# Q18123880 tennis singles
queryString = """
SELECT ?p ?pName ?o ?oName
WHERE 
{
    wd:Q18123880 ?p ?o.
    ?p sc:name ?pName .
    ?o sc:name ?oName .
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3095'), ('pName', 'practiced by'), ('o', 'http://www.wikidata.org/entity/Q10833314'), ('oName', 'tennis player')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q2312410'), ('oName', 'sports discipline')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pName', 'sport'), ('o', 'http://www.wikidata.org/entity/Q847'), ('oName', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q847'), ('oName', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q1967745'), ('oName', 'type of tennis match')]
5


In [307]:
queryString = """
SELECT DISTINCT ?p ?pName ?oName
WHERE {
   wd:Q1426 ?p ?o .
    ?p sc:name ?pName.
    #?o sc:name ?oName.
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('pName', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('pName', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('pName', 'employer')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('pName', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1280'), ('pName', 'CONOR.SI ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1284'), ('pName', 'Munzinger person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1285'), ('pName', 'Munzinger Sport number')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('pName', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pName', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1352')

In [313]:
# P1352 ranking
queryString = """
SELECT ?o 
WHERE {
   wd:Q1426 wdt:P1352 ?o .
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('o', '1')]
[('o', '24')]
2


In [316]:
queryString = """
SELECT DISTINCT ?tennisplayerName ?ranking ?conutry
WHERE {
   ?tennisplayer wdt:P106 wd:Q10833314 ;
           sc:name ?tennisplayerName ;
        wdt:P1352 ?ranking ; 
        wdt:P1532 ?conutry .
   ?conutry sc:name ?conutryName .
   FILTER (xsd:integer(?ranking) = 1)
}
LIMIT 100
"""
print("Results") 
x=run_query(queryString)

Results
[('tennisplayerName', 'Hsieh Su-wei'), ('ranking', '1'), ('conutry', 'http://www.wikidata.org/entity/Q216923')]
[('tennisplayerName', 'Steffi Graf'), ('ranking', '1'), ('conutry', 'http://www.wikidata.org/entity/Q183')]
[('tennisplayerName', 'Boris Becker'), ('ranking', '1'), ('conutry', 'http://www.wikidata.org/entity/Q183')]
[('tennisplayerName', 'Victoria Azarenka'), ('ranking', '1'), ('conutry', 'http://www.wikidata.org/entity/Q184')]
[('tennisplayerName', 'Karolína Plíšková'), ('ranking', '1'), ('conutry', 'http://www.wikidata.org/entity/Q213')]
[('tennisplayerName', 'Whitney Osuigwe'), ('ranking', '1'), ('conutry', 'http://www.wikidata.org/entity/Q30')]
[('tennisplayerName', 'Jimmy Connors'), ('ranking', '1'), ('conutry', 'http://www.wikidata.org/entity/Q30')]
[('tennisplayerName', 'Robert Seguso'), ('ranking', '1'), ('conutry', 'http://www.wikidata.org/entity/Q30')]
[('tennisplayerName', 'Ivan Lendl'), ('ranking', '1'), ('conutry', 'http://www.wikidata.org/entity/Q30')]


Final query for this task

In [317]:
# write your final query
queryString = """
SELECT DISTINCT ?conutry ?conutryName
WHERE {
   ?tennisplayer wdt:P106 wd:Q10833314 ;
           sc:name ?tennisplayerName ;
        wdt:P1352 ?ranking ; 
        wdt:P1532 ?conutry .
   ?conutry sc:name ?conutryName .
   FILTER (xsd:integer(?ranking) = 1)
}
LIMIT 100
"""
print("Results") 
x=run_query(queryString)

Results
[('conutry', 'http://www.wikidata.org/entity/Q216923'), ('conutryName', 'Chinese Taipei')]
[('conutry', 'http://www.wikidata.org/entity/Q183'), ('conutryName', 'Germany')]
[('conutry', 'http://www.wikidata.org/entity/Q184'), ('conutryName', 'Belarus')]
[('conutry', 'http://www.wikidata.org/entity/Q213'), ('conutryName', 'Czech Republic')]
[('conutry', 'http://www.wikidata.org/entity/Q30'), ('conutryName', 'United States of America')]
[('conutry', 'http://www.wikidata.org/entity/Q33'), ('conutryName', 'Finland')]
[('conutry', 'http://www.wikidata.org/entity/Q34'), ('conutryName', 'Sweden')]
[('conutry', 'http://www.wikidata.org/entity/Q36'), ('conutryName', 'Poland')]
[('conutry', 'http://www.wikidata.org/entity/Q668'), ('conutryName', 'India')]
[('conutry', 'http://www.wikidata.org/entity/Q33946'), ('conutryName', 'Czechoslovakia')]
[('conutry', 'http://www.wikidata.org/entity/Q159'), ('conutryName', 'Russia')]
[('conutry', 'http://www.wikidata.org/entity/Q29'), ('conutryName',